In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geemap
import ee

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:

Map = geemap.Map()
geometry = ee.Geometry.Point([77.40600099670314, 23.250822912946425])
image = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(geometry) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .first()
start_date, end_date = "2010-01-01", "2020-01-01"

ndwi = image.normalizedDifference(['B3', 'B8'])

ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1 = ndwiMasked.toInt()

vectors = ndwiMasked1.reduceToVectors(
    scale=30.0, geometryType='polygon', eightConnected=False, maxPixels=10000000, bestEffort=True)


In [4]:
geometry = ee.Geometry.Point([77.42174899777628, 23.205021037202645])
Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

visualization = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
Map.addLayer(sentinel, visualization, 'sent2rgb')

mndwi = sentinel.normalizedDifference(['B3', 'B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11', 'B12']).rename('ndsi')
ndsi2 = sentinel.normalizedDifference(
    ['B11', 'B12']).rename('ndsi2').mask(mndwitr)
Map.addLayer(ndsi2, {'min': 0.1, 'max': 0.4, 'palette': [
             'cyan', 'orange', 'red']}, 'salinity')
ndti = sentinel.normalizedDifference(['B4', 'B3']).rename('ndti')
ndti2 = sentinel.normalizedDifference(
    ['B4', 'B3']).rename('ndti2').mask(mndwitr)
Map.addLayer(ndti2, {'min': -1, 'max': +1,
             'palette': ['blue', 'pink', 'brown']}, 'turbidity')

ndci = sentinel.normalizedDifference(['B5', 'B4']).rename('ndci')
ndci2 = sentinel.normalizedDifference(
    ['B5', 'B4']).rename('ndci2').mask(mndwitr)
Map.addLayer(ndci2, {'min': -1, 'max': +1,
             'palette': ['green', 'pink', 'brown']}, 'chlorophyll')

#nitrate = sentinel.expression('(B4 + B3) / (B4 - B3)', {
    #'B3': sentinel.select('B3'),
    #'B4': sentinel.select('B4')}).rename('nitrate')
#Map.addLayer(nitrate, {'min': -1, 'max': 1,'palette': ['blue', 'white', 'green']}, 'nitrate')

#phosphate = sentinel.expression('(B4 - B5) / (B4 + B5)', {
    #'B4': sentinel.select('B4'),
    #'B5': sentinel.select('B5')}).rename('phosphate')
#Map.addLayer(phosphate, {'min': -1, 'max': 1,'palette': ['brown', 'white', 'green']}, 'phosphate')


ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(
    sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')
ph2 = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select(
    'B1').divide(sentinel.select('B8')))).rename('ph2').mask(mndwitr)
Map.addLayer(ph2, {'min': 0, 'max': 14, 'palette': [
             'red', 'yellow', 'cyan']}, 'ph')

dissolvedoxygen = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select(
    'B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')
dissolvedoxygen2 = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(
    ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen2').mask(mndwitr)
Map.addLayer(dissolvedoxygen2, {'min': 6.5, 'max': 8, 'palette': [
             'red', 'green', 'blue']}, 'do')



In [5]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filterBounds(vectors).median()

temp = col.select(
    'ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')


In [6]:
starting = '2021-01-01'
ending = '2021-08-31'

data = ee.ImageCollection(
    'COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
    .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)


In [7]:
dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa04_radiance')).rename('dom')
dom2 = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa04_radiance')).mask(mndwitr)
Map.addLayer(dom2, {'min': 0, 'max': 0.8, 'palette': [
             'green', 'red', 'yellow']}, 'Dissolved organic matter')
suspended_matter_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa06_radiance')).rename('suspended_matter')
suspended_matter2 = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa06_radiance')).mask(mndwitr)
Map.addLayer(suspended_matter2, {'min': 0, 'max': 0.8, 'palette': [
    'green', 'red', 'yellow']}, 'suspended_matter')


In [8]:
latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)
data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))



In [9]:
latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)

data_sm_2021_Jan_August_test = np.array(
    (ee.Array(latlon.get("suspended_matter")).getInfo()))


In [10]:
latlon = ee.Image.pixelLonLat().addBands(temp)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)

data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))


In [11]:
latlon = ee.Image.pixelLonLat().addBands(ndti)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))


In [12]:
latlon = ee.Image.pixelLonLat().addBands(ndsi)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))


In [13]:
latlon = ee.Image.pixelLonLat().addBands(ndci)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))


In [14]:
latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)

data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))


In [15]:
latlon = ee.Image.pixelLonLat().addBands(ph)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))


In [16]:
'''latlon = ee.Image.pixelLonLat().addBands(nitrate)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_nitrate = np.array((ee.Array(latlon.get("nitrate")).getInfo()))'''


'latlon = ee.Image.pixelLonLat().addBands(nitrate)\n\nlatlon = latlon.reduceRegion(\n    reducer=ee.Reducer.toList(),\n    geometry=vectors,\n    scale=100)\ndata_nitrate = np.array((ee.Array(latlon.get("nitrate")).getInfo()))'

In [17]:
'''latlon = ee.Image.pixelLonLat().addBands(phosphate)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_phosphate = np.array((ee.Array(latlon.get("phosphate")).getInfo()))'''


'latlon = ee.Image.pixelLonLat().addBands(phosphate)\n\nlatlon = latlon.reduceRegion(\n    reducer=ee.Reducer.toList(),\n    geometry=vectors,\n    scale=100)\ndata_phosphate = np.array((ee.Array(latlon.get("phosphate")).getInfo()))'

In [18]:
df = pd.concat([pd.DataFrame(data_do, columns=['Dissolved Oxygen']),
                pd.DataFrame(data_ndsi, columns=['Salinity']),
                pd.DataFrame(data_lst, columns=['Temperature']),
                pd.DataFrame(data_ph, columns=['pH']),
                pd.DataFrame(data_ndti, columns=['Turbidity']),
                pd.DataFrame(data_dom_2021_Jan_August_test,
                             columns=['Dissolved Organic Matter']),
                pd.DataFrame(data_sm_2021_Jan_August_test,
                             columns=['Suspended Matter']),         
                pd.DataFrame(data_ndci, columns=['Chlorophyll'])], axis=1, sort=False)



In [19]:
df

,Dissolved Oxygen,Salinity,Temperature,pH,Turbidity,Dissolved Organic Matter,Suspended Matter,Chlorophyll
0,0.769450,0.148970,27.614361,8.000461,-0.078048,0.523666,0.617774,0.166928
1,6.565650,0.219250,26.781028,8.080300,-0.109098,0.523388,0.633314,0.257878
2,7.906200,0.283740,26.738493,8.105265,-0.110585,0.522102,0.615449,0.269373
3,7.814250,0.093366,30.193352,7.979497,0.029447,0.523885,0.567422,0.060146
4,8.037150,0.112670,29.437020,7.908079,0.029490,0.512813,0.566820,0.065705
...,...,...,...,...,...,...,...,...
13279,0.258408,0.110887,27.718000,7.995021,0.046290,0.467905,0.579077,0.089534
13280,5.248033,0.139577,26.715592,8.014529,0.016798,0.460299,0.533431,0.109141
13281,10.250499,0.116486,26.762989,8.015362,0.072047,0.474616,0.536491,0.111274
13282,6.282130,0.110908,26.686881,7.990602,-0.033525,0.462093,0.537560,0.125063


In [20]:
df.to_csv('dataset.csv')

In [21]:
df.corr()

,Dissolved Oxygen,Salinity,Temperature,pH,Turbidity,Dissolved Organic Matter,Suspended Matter,Chlorophyll
Dissolved Oxygen,1.000000,-0.046318,0.004869,0.009540,0.086697,-0.031620,0.050724,-0.086185
Salinity,-0.046318,1.000000,-0.139905,0.026860,-0.285260,0.026622,0.074866,0.251065
Temperature,0.004869,-0.139905,1.000000,0.392518,0.429862,-0.033765,-0.053102,0.137840
pH,0.009540,0.026860,0.392518,1.000000,0.813689,0.130180,0.104212,0.797305
Turbidity,0.086697,-0.285260,0.429862,0.813689,1.000000,0.141379,0.113717,0.451102
Dissolved Organic Matter,-0.031620,0.026622,-0.033765,0.130180,0.141379,1.000000,0.402485,0.110006
Suspended Matter,0.050724,0.074866,-0.053102,0.104212,0.113717,0.402485,1.000000,0.062837
Chlorophyll,-0.086185,0.251065,0.137840,0.797305,0.451102,0.110006,0.062837,1.000000
